In [ ]:
%matplotlib inline
import os
import sys
import numpy as np

sys.path.append("../../")

from sapsan.lib.backends import MLflowBackend
from sapsan.lib.data import HDF5Dataset, EquidistantSampling, flatten
from sapsan.lib.estimator.krr.krr_estimator import KRR, KRRConfig
from sapsan import Train, Evaluate, model_graph

In [ ]:
#--- Experiment tracking backend ---
#MLflow - the server will be launched automatically
#in case it won't, type in cmd: mlflow ui --port=9000
#uncomment tracking_backend to use mlflow

experiment_name = "KRR experiment"
#tracking_backend = MLflowBackend(experiment_name, host="localhost", port=9000)

In [ ]:
#--- Data setup ---
#In the intereset of loading and training multiple timesteps
#one can specify which checkpoints to use and where
#they appear in the path via syntax: {checkpoint:format}
#
#Next, you need to specify which features to load; let's assume 
#        path = "{feature}.h5"
#
# 1) If in different files, then specify features directly;
#    The default HDF5 label will be the last label in the file
#    Ex: features = ['velocity', 'denisty', 'pressure']
# 2) If in the same, then duplicate the name in features
#    and specify which labels to pull
#    Ex: features = ["data", "data", "data"]
#        feature_labels = ['velocity', 'density', 'pressure']

path = 'data/cnn_krr/t{checkpoint:1.0f}/{feature}_dim128_2d.h5'
features = ['u']
target = ['u']


#Dimensionality of your data in format [D,H,W]
INPUT_SIZE = [128,128]

#Reduce dimensionality to the following in format [D,H,W]
SAMPLE_TO = [64,64]

#Sampler to use for reduction
sampler = EquidistantSampling(SAMPLE_TO)

In [ ]:
#Load the data
data_loader = HDF5Dataset(path=path,
                   features=features,
                   target=target,
                   checkpoints=[0],
                   input_size=INPUT_SIZE,
                   sampler=sampler, 
                   flat = True)
x, y = data_loader.load_numpy()

#ex: predict one component of the target
y = np.array([y[0]])

loaders = [x,y]

In [ ]:
#Machine Learning model to use

#Configuration of the model parameters:
#    alpha - rbf kernel hyperparameter
#    gamma - rbf kernel hyperparameter
estimator = KRR(config = KRRConfig(gamma=1.778, alpha=0.001, kernel='rbf'),
                loaders = loaders)

In [ ]:
#--- Train the model ---
#Set the experiment
training_experiment = Train(model = estimator,
                            #backend=tracking_backend, #uncomment to use mlflow                            
                            data_parameters = data_loader)
#Train the model
estimator = training_experiment.run()

In [ ]:
%matplotlib inline

#--- Test the model ---
#Load the test data
data_loader = HDF5Dataset(path=path,
                   features=features,
                   target=target,
                   checkpoints=[0],
                   input_size=INPUT_SIZE,
                   sampler=sampler, 
                   flat = True)
x, y = data_loader.load_numpy()

#ex: predict one component of the target
y = np.array([y[0]])
loaders = [x,y]

#Set the test experiment
estimator.loaders = loaders
evaluation_experiment = Evaluate(model = estimator,
                                 #backend=tracking_backend, #uncomment to use mlflow
                                 data_parameters = data_loader)

#Test the model
#eval returns a dict, cubes = {'pred_cube':np.ndarray, 'target_cube':np.ndarray}
cubes = evaluation_experiment.run()